In [ ]:
# drive mount. colab에 내 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
import json
import pickle
import numpy as np

In [ ]:
# GPU 찾기. 없으면 CPU로 동작
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# check torch is available
print(torch.__version__)
print(torch.tensor([1.0, 2.0]).cuda())

In [ ]:
def combine_subtokens(tokens, relevance_score):
    '''
    combine subtokens to one word
    '''

    token = ['[CLS]', '[SEP]', '[MASK]', '[UNK]', '[PAD]']

    word, score, num = [], [], []

    for i in range(len(tokens)):
      if tokens[i] in token: continue
      if tokens[i][0]=='#':
        word[-1]+=tokens[i][2:]
        score[-1]+=relevance_score[i]
        num[-1]+=1
      else:
        word.append(tokens[i])
        score.append(relevance_score[i])
        num.append(1)
    
    for i in range(len(word)): score[i]/=num[i]
    
    return word, score


In [ ]:
def get_output(string,tokenizer,model):  
  encoded_dict = tokenizer.encode_plus(
                      string, 
                      add_special_tokens = True,
                      max_length = 512,
                      truncation=True,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_tensors = 'pt',
                )
  input_ids = encoded_dict['input_ids'].to(device)
  attention_masks = encoded_dict['attention_mask'].to(device)
  output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_masks)[0], dim=-1).detach().cpu().numpy()
  return output


def get_comp_suff(tokens, expl, tokenizer, model, rationale_num):
  tokens_comb, expl_comb = combine_subtokens(tokens, list(expl))

  top_score = [(expl_comb[i],i) for i in range(len(expl_comb))]
  top_score.sort(reverse=True)
  top_score_idx = set(i for _,i in top_score[:rationale_num])

  no_rationale_tokens = []
  rationale_tokens = []
  for i, token in enumerate(tokens_comb):
    if i in top_score_idx: rationale_tokens.append(token)
    else: no_rationale_tokens.append(token)
  
  orig_string = tokenizer.convert_tokens_to_string(tokens_comb)
  no_rationale_string = tokenizer.convert_tokens_to_string(no_rationale_tokens)
  rationale_string = tokenizer.convert_tokens_to_string(rationale_tokens)

  orig_output = get_output(orig_string,tokenizer,model)[0][1]
  no_rationale_output = get_output(no_rationale_string,tokenizer,model)[0][1]
  rationale_output = get_output(rationale_string,tokenizer,model)[0][1]
  
  comp = orig_output-no_rationale_output
  suff = orig_output-rationale_output
  return comp, suff


def get_comp_suff_all(target_model_name, expl_name, rationale_num = 3,data_num = 5):
  root = "/content/drive/MyDrive/CS470_team_2in1/colab"

  records = np.load(root+'/explanation/balanced_only1000model/'+expl_name, allow_pickle=True)

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

  target_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
  target_model.load_state_dict(torch.load(root+'/model/'+target_model_name, map_location=device))
  target_model.cuda()

  total_comp = 0
  total_suff = 0
  num = 0
  for tokens, rating, product, output, neg_expl, pos_expl, true_class, pred_class in records:
    if num==data_num: break
    if not(len(tokens)==len(neg_expl)==len(pos_expl)): continue
    if len(combine_subtokens(tokens, list(pos_expl))[0])<rationale_num*2: continue

    comp, suff = get_comp_suff(tokens,pos_expl,tokenizer,target_model,rationale_num)
    total_comp+=comp
    total_suff+=suff

    num+=1
  
  total_comp/=data_num
  total_suff/=data_num

  return total_comp,total_suff

In [ ]:
comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name='amazon_book_expl-transformer_attribution-binaryclassification_only10000.npy', rationale_num = 5,data_num = 100)
print(comp,suff)

In [ ]:
comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name='amazon_book_expl-transformer_attribution-binaryclassification_notrained_only10000.npy', rationale_num = 5,data_num = 100)
print(comp,suff)

In [ ]:
comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name='amazon_book_expl-transformer_attribution-regression_only10000.npy', rationale_num = 5,data_num = 100)
print(comp,suff)

In [ ]:
comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name='amazon_book_expl-partial_lrp-regression_only10000.npy', rationale_num = 5,data_num = 100)
print(comp,suff)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
comp_list, suff_list = [], []
binary_expl_npy = 'amazon_book_expl-transformer_attribution-binaryclassification_only10000.npy'
regression_expl_npy = 'amazon_book_expl-transformer_attribution-regression_only10000.npy'
for rat_num in tqdm(range(1, 100, 2)):
    comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name=binary_expl_npy, rationale_num = rat_num,data_num = 100)
    # print(comp,suff)
    comp_list.append(comp)
    suff_list.append(suff)


In [ ]:
reg_comp_list, reg_suff_list = [], []
binary_expl_npy = 'amazon_book_expl-transformer_attribution-binaryclassification_only10000.npy'
regression_expl_npy = 'amazon_book_expl-transformer_attribution-regression_only10000.npy'
for rat_num in tqdm(range(1, 100, 2)):
    comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name=regression_expl_npy, rationale_num = rat_num,data_num = 100)
    # print(comp,suff)
    reg_comp_list.append(comp)
    reg_suff_list.append(suff)

In [ ]:
not_comp_list, not_suff_list = [], []
binary_expl_npy = 'amazon_book_expl-transformer_attribution-binaryclassification_only10000.npy'
regression_expl_npy = 'amazon_book_expl-transformer_attribution-regression_only10000.npy'
notrained_expl_npy = 'amazon_book_expl-transformer_attribution-binaryclassification_notrained_only10000.npy'
for rat_num in tqdm(range(1, 100, 2)):
    comp,suff = get_comp_suff_all(target_model_name='amazon_book_only1000_binaryclassification_testacc097.pt', expl_name=notrained_expl_npy, rationale_num = rat_num,data_num = 100)
    # print(comp,suff)
    not_comp_list.append(comp)
    not_suff_list.append(suff)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot([2*i+1 for i in range(len(comp_list))], comp_list, label='binary')
plt.plot([2*i+1 for i in range(len(reg_comp_list))], reg_comp_list, label='regression')
plt.plot([2*i+1 for i in range(len(not_comp_list))], not_comp_list, label='not trained')
plt.title('comprehensiveness')
plt.xlabel('# of rationales')
plt.legend(('binary', 'regression', 'not trained'))
# plt.ylabel('comprehensiveness')

plt.subplot(1, 2, 2)
plt.plot([2*i+1 for i in range(len(suff_list))], suff_list, label='sufficiency')
plt.plot([2*i+1 for i in range(len(reg_suff_list))], reg_suff_list, label='regression')
plt.plot([2*i+1 for i in range(len(not_suff_list))], not_suff_list, label='not trained')
plt.title('sufficiency')
plt.xlabel('# of rationales')
plt.legend(('binary', 'regression', 'not trained')
# plt.ylabel('sufficiency')
# plt.legend(('train', 'val'))
plt.savefig(f'comp_suff_1.png')
plt.show()